In [7]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs('images', exist_ok=True)
os.makedirs('saved_models', exist_ok=True)

channels = 1
img_size = 28
n_classes = 10
latent_dim = 100
batch_size = 32
learning_rate = .0002
b1 = .5
b2 = .999
sample_interval = 1

img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [  nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim+n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(n_classes, n_classes)
        
        self.model = nn.Sequential(
            nn.Linear(n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1)
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity
    
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

# Loss functions
adversarial_loss = torch.nn.MSELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [12]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row**2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, 'images/%d.png' % batches_done, nrow=n_row, normalize=True)

In [13]:
class QuickDrawDataset(Dataset):
    """Quick Draw dataset."""

    def __init__(self, label, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = np.load(f'data/{label}.npy')
        self.label = label
        self.transform = transform
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        image = self.data_frame[idx]
        label = self.label
        return image, 1

In [14]:
data = QuickDrawDataset(label='panda', transform=transforms.Compose([
                        transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
dataloader = DataLoader(data, batch_size=32, shuffle=False)

In [15]:
for epoch in range(1):
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = len(imgs)
        
        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)
        
        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor)) # 32x794
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, 1, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch, 1, i, len(dataloader),
                                                            d_loss.item(), g_loss.item()))

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)


[Epoch 0/1] [Batch 0/3551] [D loss: 83.517715] [G loss: 1.003725]
[Epoch 0/1] [Batch 1/3551] [D loss: 52.275719] [G loss: 1.031304]
[Epoch 0/1] [Batch 2/3551] [D loss: 19.278305] [G loss: 1.021687]
[Epoch 0/1] [Batch 3/3551] [D loss: 14.314573] [G loss: 1.012913]
[Epoch 0/1] [Batch 4/3551] [D loss: 24.394438] [G loss: 1.013031]
[Epoch 0/1] [Batch 5/3551] [D loss: 12.876037] [G loss: 1.005726]
[Epoch 0/1] [Batch 6/3551] [D loss: 17.008179] [G loss: 1.012735]
[Epoch 0/1] [Batch 7/3551] [D loss: 13.743367] [G loss: 0.999223]
[Epoch 0/1] [Batch 8/3551] [D loss: 21.078283] [G loss: 1.003845]
[Epoch 0/1] [Batch 9/3551] [D loss: 8.406813] [G loss: 1.000113]
[Epoch 0/1] [Batch 10/3551] [D loss: 8.706357] [G loss: 1.003444]
[Epoch 0/1] [Batch 11/3551] [D loss: 12.011793] [G loss: 1.003147]
[Epoch 0/1] [Batch 12/3551] [D loss: 12.344916] [G loss: 0.996042]
[Epoch 0/1] [Batch 13/3551] [D loss: 7.106085] [G loss: 0.993984]
[Epoch 0/1] [Batch 14/3551] [D loss: 6.892169] [G loss: 0.991791]
[Epoch 0/

[Epoch 0/1] [Batch 125/3551] [D loss: 1.494734] [G loss: 0.556255]
[Epoch 0/1] [Batch 126/3551] [D loss: 1.230061] [G loss: 0.602888]
[Epoch 0/1] [Batch 127/3551] [D loss: 0.992658] [G loss: 0.592320]
[Epoch 0/1] [Batch 128/3551] [D loss: 1.004657] [G loss: 0.595488]
[Epoch 0/1] [Batch 129/3551] [D loss: 1.423864] [G loss: 0.633839]
[Epoch 0/1] [Batch 130/3551] [D loss: 2.265957] [G loss: 0.612206]
[Epoch 0/1] [Batch 131/3551] [D loss: 1.013640] [G loss: 0.627333]
[Epoch 0/1] [Batch 132/3551] [D loss: 1.242804] [G loss: 0.635159]
[Epoch 0/1] [Batch 133/3551] [D loss: 1.877136] [G loss: 0.631804]
[Epoch 0/1] [Batch 134/3551] [D loss: 1.456555] [G loss: 0.622109]
[Epoch 0/1] [Batch 135/3551] [D loss: 1.046024] [G loss: 0.607634]
[Epoch 0/1] [Batch 136/3551] [D loss: 1.699797] [G loss: 0.640790]
[Epoch 0/1] [Batch 137/3551] [D loss: 1.282310] [G loss: 0.600635]
[Epoch 0/1] [Batch 138/3551] [D loss: 1.063629] [G loss: 0.620258]
[Epoch 0/1] [Batch 139/3551] [D loss: 1.444834] [G loss: 0.629

[Epoch 0/1] [Batch 248/3551] [D loss: 0.656189] [G loss: 0.740933]
[Epoch 0/1] [Batch 249/3551] [D loss: 0.757444] [G loss: 0.771180]
[Epoch 0/1] [Batch 250/3551] [D loss: 0.727734] [G loss: 0.747837]
[Epoch 0/1] [Batch 251/3551] [D loss: 0.712137] [G loss: 0.814939]
[Epoch 0/1] [Batch 252/3551] [D loss: 0.488173] [G loss: 0.797995]
[Epoch 0/1] [Batch 253/3551] [D loss: 0.743201] [G loss: 0.772696]
[Epoch 0/1] [Batch 254/3551] [D loss: 0.698039] [G loss: 0.801273]
[Epoch 0/1] [Batch 255/3551] [D loss: 0.570249] [G loss: 0.803734]
[Epoch 0/1] [Batch 256/3551] [D loss: 0.846992] [G loss: 0.784910]
[Epoch 0/1] [Batch 257/3551] [D loss: 1.073954] [G loss: 0.778411]
[Epoch 0/1] [Batch 258/3551] [D loss: 0.838016] [G loss: 0.836499]
[Epoch 0/1] [Batch 259/3551] [D loss: 0.676630] [G loss: 0.793436]
[Epoch 0/1] [Batch 260/3551] [D loss: 0.496900] [G loss: 0.789631]
[Epoch 0/1] [Batch 261/3551] [D loss: 0.438696] [G loss: 0.834114]
[Epoch 0/1] [Batch 262/3551] [D loss: 0.420588] [G loss: 0.820

[Epoch 0/1] [Batch 372/3551] [D loss: 0.414024] [G loss: 0.915581]
[Epoch 0/1] [Batch 373/3551] [D loss: 0.375620] [G loss: 0.894849]
[Epoch 0/1] [Batch 374/3551] [D loss: 0.523633] [G loss: 0.858353]
[Epoch 0/1] [Batch 375/3551] [D loss: 0.329586] [G loss: 0.838696]
[Epoch 0/1] [Batch 376/3551] [D loss: 0.562373] [G loss: 0.865649]
[Epoch 0/1] [Batch 377/3551] [D loss: 0.477555] [G loss: 0.885907]
[Epoch 0/1] [Batch 378/3551] [D loss: 0.492359] [G loss: 0.836455]
[Epoch 0/1] [Batch 379/3551] [D loss: 0.513388] [G loss: 0.872218]
[Epoch 0/1] [Batch 380/3551] [D loss: 0.437003] [G loss: 0.870367]
[Epoch 0/1] [Batch 381/3551] [D loss: 0.285485] [G loss: 0.873066]
[Epoch 0/1] [Batch 382/3551] [D loss: 0.366669] [G loss: 0.889467]
[Epoch 0/1] [Batch 383/3551] [D loss: 0.460201] [G loss: 0.857388]
[Epoch 0/1] [Batch 384/3551] [D loss: 0.467596] [G loss: 0.860646]
[Epoch 0/1] [Batch 385/3551] [D loss: 0.495941] [G loss: 0.881912]
[Epoch 0/1] [Batch 386/3551] [D loss: 0.319133] [G loss: 0.890

[Epoch 0/1] [Batch 496/3551] [D loss: 0.207098] [G loss: 0.869167]
[Epoch 0/1] [Batch 497/3551] [D loss: 0.191944] [G loss: 0.918323]
[Epoch 0/1] [Batch 498/3551] [D loss: 0.308196] [G loss: 0.887639]
[Epoch 0/1] [Batch 499/3551] [D loss: 0.367328] [G loss: 0.886140]
[Epoch 0/1] [Batch 500/3551] [D loss: 0.447948] [G loss: 0.878551]
[Epoch 0/1] [Batch 501/3551] [D loss: 0.245521] [G loss: 0.925116]
[Epoch 0/1] [Batch 502/3551] [D loss: 0.375402] [G loss: 0.865255]
[Epoch 0/1] [Batch 503/3551] [D loss: 0.335586] [G loss: 0.915842]
[Epoch 0/1] [Batch 504/3551] [D loss: 0.359277] [G loss: 0.876409]
[Epoch 0/1] [Batch 505/3551] [D loss: 0.338081] [G loss: 0.894884]
[Epoch 0/1] [Batch 506/3551] [D loss: 0.317892] [G loss: 0.856275]
[Epoch 0/1] [Batch 507/3551] [D loss: 0.240193] [G loss: 0.892523]
[Epoch 0/1] [Batch 508/3551] [D loss: 0.231654] [G loss: 0.926954]
[Epoch 0/1] [Batch 509/3551] [D loss: 0.243545] [G loss: 0.879392]
[Epoch 0/1] [Batch 510/3551] [D loss: 0.275789] [G loss: 0.893

...

[Epoch 0/1] [Batch 2949/3551] [D loss: 0.040586] [G loss: 0.980000]
[Epoch 0/1] [Batch 2950/3551] [D loss: 0.021086] [G loss: 0.977498]
[Epoch 0/1] [Batch 2951/3551] [D loss: 0.021715] [G loss: 0.974206]
[Epoch 0/1] [Batch 2952/3551] [D loss: 0.024550] [G loss: 0.987580]
[Epoch 0/1] [Batch 2953/3551] [D loss: 0.026728] [G loss: 0.986230]
[Epoch 0/1] [Batch 2954/3551] [D loss: 0.044944] [G loss: 0.984256]
[Epoch 0/1] [Batch 2955/3551] [D loss: 0.035422] [G loss: 0.979987]
[Epoch 0/1] [Batch 2956/3551] [D loss: 0.025386] [G loss: 0.991277]
[Epoch 0/1] [Batch 2957/3551] [D loss: 0.028321] [G loss: 0.988235]
[Epoch 0/1] [Batch 2958/3551] [D loss: 0.024419] [G loss: 0.987185]
[Epoch 0/1] [Batch 2959/3551] [D loss: 0.042761] [G loss: 0.992422]
[Epoch 0/1] [Batch 2960/3551] [D loss: 0.026715] [G loss: 0.981426]
[Epoch 0/1] [Batch 2961/3551] [D loss: 0.028618] [G loss: 0.988262]
[Epoch 0/1] [Batch 2962/3551] [D loss: 0.022932] [G loss: 0.978487]
[Epoch 0/1] [Batch 2963/3551] [D loss: 0.019620]

[Epoch 0/1] [Batch 3071/3551] [D loss: 0.028683] [G loss: 0.984249]
[Epoch 0/1] [Batch 3072/3551] [D loss: 0.020165] [G loss: 0.980717]
[Epoch 0/1] [Batch 3073/3551] [D loss: 0.045409] [G loss: 0.981708]
[Epoch 0/1] [Batch 3074/3551] [D loss: 0.017595] [G loss: 0.981155]
[Epoch 0/1] [Batch 3075/3551] [D loss: 0.025312] [G loss: 0.971439]
[Epoch 0/1] [Batch 3076/3551] [D loss: 0.021952] [G loss: 0.988407]
[Epoch 0/1] [Batch 3077/3551] [D loss: 0.030359] [G loss: 0.978659]
[Epoch 0/1] [Batch 3078/3551] [D loss: 0.022185] [G loss: 0.969867]
[Epoch 0/1] [Batch 3079/3551] [D loss: 0.021209] [G loss: 0.973828]
[Epoch 0/1] [Batch 3080/3551] [D loss: 0.017061] [G loss: 0.975739]
[Epoch 0/1] [Batch 3081/3551] [D loss: 0.047492] [G loss: 0.975054]
[Epoch 0/1] [Batch 3082/3551] [D loss: 0.033609] [G loss: 0.985183]
[Epoch 0/1] [Batch 3083/3551] [D loss: 0.046929] [G loss: 0.979153]
[Epoch 0/1] [Batch 3084/3551] [D loss: 0.025076] [G loss: 0.979494]
[Epoch 0/1] [Batch 3085/3551] [D loss: 0.021093]

[Epoch 0/1] [Batch 3194/3551] [D loss: 0.025153] [G loss: 0.985474]
[Epoch 0/1] [Batch 3195/3551] [D loss: 0.019424] [G loss: 0.983867]
[Epoch 0/1] [Batch 3196/3551] [D loss: 0.027259] [G loss: 0.976348]
[Epoch 0/1] [Batch 3197/3551] [D loss: 0.018895] [G loss: 0.969808]
[Epoch 0/1] [Batch 3198/3551] [D loss: 0.032107] [G loss: 0.972738]
[Epoch 0/1] [Batch 3199/3551] [D loss: 0.024631] [G loss: 0.978126]
[Epoch 0/1] [Batch 3200/3551] [D loss: 0.034788] [G loss: 0.974909]
[Epoch 0/1] [Batch 3201/3551] [D loss: 0.020263] [G loss: 0.983109]
[Epoch 0/1] [Batch 3202/3551] [D loss: 0.023654] [G loss: 0.982890]
[Epoch 0/1] [Batch 3203/3551] [D loss: 0.017787] [G loss: 0.971471]
[Epoch 0/1] [Batch 3204/3551] [D loss: 0.054601] [G loss: 0.978126]
[Epoch 0/1] [Batch 3205/3551] [D loss: 0.018887] [G loss: 0.980836]
[Epoch 0/1] [Batch 3206/3551] [D loss: 0.025993] [G loss: 0.984224]
[Epoch 0/1] [Batch 3207/3551] [D loss: 0.010194] [G loss: 0.987515]
[Epoch 0/1] [Batch 3208/3551] [D loss: 0.027338]

[Epoch 0/1] [Batch 3315/3551] [D loss: 0.028016] [G loss: 0.981144]
[Epoch 0/1] [Batch 3316/3551] [D loss: 0.026365] [G loss: 0.972915]
[Epoch 0/1] [Batch 3317/3551] [D loss: 0.034938] [G loss: 0.982694]
[Epoch 0/1] [Batch 3318/3551] [D loss: 0.046454] [G loss: 0.980873]
[Epoch 0/1] [Batch 3319/3551] [D loss: 0.024508] [G loss: 0.986972]
[Epoch 0/1] [Batch 3320/3551] [D loss: 0.022265] [G loss: 0.981737]
[Epoch 0/1] [Batch 3321/3551] [D loss: 0.041419] [G loss: 0.992406]
[Epoch 0/1] [Batch 3322/3551] [D loss: 0.045416] [G loss: 0.989479]
[Epoch 0/1] [Batch 3323/3551] [D loss: 0.026219] [G loss: 0.980128]
[Epoch 0/1] [Batch 3324/3551] [D loss: 0.018750] [G loss: 0.972111]
[Epoch 0/1] [Batch 3325/3551] [D loss: 0.016355] [G loss: 0.966149]
[Epoch 0/1] [Batch 3326/3551] [D loss: 0.027042] [G loss: 0.981969]
[Epoch 0/1] [Batch 3327/3551] [D loss: 0.027814] [G loss: 0.976784]
[Epoch 0/1] [Batch 3328/3551] [D loss: 0.024376] [G loss: 0.973093]
[Epoch 0/1] [Batch 3329/3551] [D loss: 0.029665]

[Epoch 0/1] [Batch 3436/3551] [D loss: 0.029679] [G loss: 0.981486]
[Epoch 0/1] [Batch 3437/3551] [D loss: 0.023073] [G loss: 0.977968]
[Epoch 0/1] [Batch 3438/3551] [D loss: 0.018779] [G loss: 0.982092]
[Epoch 0/1] [Batch 3439/3551] [D loss: 0.027989] [G loss: 0.988923]
[Epoch 0/1] [Batch 3440/3551] [D loss: 0.023976] [G loss: 0.983960]
[Epoch 0/1] [Batch 3441/3551] [D loss: 0.033306] [G loss: 0.993209]
[Epoch 0/1] [Batch 3442/3551] [D loss: 0.028473] [G loss: 0.993029]
[Epoch 0/1] [Batch 3443/3551] [D loss: 0.032835] [G loss: 0.981339]
[Epoch 0/1] [Batch 3444/3551] [D loss: 0.040709] [G loss: 0.985528]
[Epoch 0/1] [Batch 3445/3551] [D loss: 0.035225] [G loss: 0.983149]
[Epoch 0/1] [Batch 3446/3551] [D loss: 0.029992] [G loss: 0.982123]
[Epoch 0/1] [Batch 3447/3551] [D loss: 0.029461] [G loss: 0.985182]
[Epoch 0/1] [Batch 3448/3551] [D loss: 0.019005] [G loss: 0.980637]
[Epoch 0/1] [Batch 3449/3551] [D loss: 0.024844] [G loss: 0.978543]
[Epoch 0/1] [Batch 3450/3551] [D loss: 0.023676]

In [39]:
def save_model():
    torch.save({
            'epoch': epoch,
            'generator': generator.model.state_dict(),
            'discriminator': discriminator.model.state_dict(),
            'optimizer_gen': optimizer_G.state_dict(),
            'optimizer_dis': optimizer_D.state_dict(),
            'G_loss': g_loss.item(),
            'D_loss': d_loss.item()
            }, 'saved_models/models')

In [30]:
def load_model(model, path):
    m = model
    m.load_state_dict(torch.load(path), strict=False)
    return m.eval()